In [1]:
import tsdm
import numpy as np

In [2]:
x = tsdm.load_dataset('electricity')
x

In [3]:
observed = np.random.choice([True, False], size=x.shape)
x = x.where(observed)
display(observed, x)

### 2.1 Triplet Format

The data is represented as a set of triplets (time, variable, value). All NaNs are dropped.

In [4]:
dense_x = tsdm.make_dense_triplets(x)
dense_x

### 2.2 Sparse Triplet format

The same as before, but the variable tensor is encoded in a one-hot fashion, and the tensor is stored as a sparse tensor

In [5]:
sparse_x = tsdm.make_sparse_triplets(x)
sparse_x

### 2.3 Masked Format

Here we get 3 tensors:

- x: the original data
- m: a boolean mask, 1: value observed, 0: value not observed (NaN)
- d: time since the channel was last observed

In [6]:
x, m, d = tsdm.make_masked_format(x)
display(x, m, d)

## 3. Visualizing the data

In [6]:
import tsdm
import numpy as np
from pandas import NA, DataFrame, testing
x = [[47, 49, NA, 40, NA, 43, 55],
     [NA, 15, 14, NA, NA, NA, 15]]
t = [0, 0.1, 0.6, 1.6, 2.2, 2.5, 3.1]

d = [[0.0, 0.1, 0.5, 1.5, 0.6, 0.9, 0.6],
     [0.0, 0.1, 0.5, 1.0, 1.6, 1.9, 2.5]]

m = [[1, 1, 0, 1, 0, 1, 1],
     [0, 1, 1, 0, 0, 0, 1]]

data = DataFrame(x, columns=t).T
mask = DataFrame(m, columns=t).T
diff = DataFrame(d, columns=t).T

x, m, d = tsdm.make_masked_format(data)
testing.assert_frame_equal(x, data, check_dtype=False)
testing.assert_frame_equal(m, mask, check_dtype=False)
testing.assert_frame_equal(d, diff, check_dtype=False)

In [10]:
x = np.array([[47, 49, np.nan, 40, np.nan, 43, 55],
              [np.nan, 15, 14, np.nan, ]] )

In [11]:
pd.NA